In [1]:
import os
import cartopy as ccrs
import numpy as np
import xarray as xr
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
from mpl_toolkits.basemap import Basemap

In [2]:
# Converting lat/long to cartesian

def get_cartesian(lat=None,lon=None, R=6371): # R is radius of the earth
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R *np.sin(lat)
    return x,y,z

# Get countrylines

In [3]:
# Function generating coastline lon/lat 
def get_coastline_traces(m):
    poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    cc_lons, cc_lats= polygons_to_traces(poly_paths, N_poly, m)
    return cc_lons, cc_lats

In [4]:
# Function generating country lon/lat 
def get_country_traces(m):
    poly_paths = m.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    country_lons, country_lats= polygons_to_traces(poly_paths, N_poly, m)
    return country_lons, country_lats

In [5]:
# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly, m):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    # init. plotting list
    lons=[]
    lats=[]

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
    
        
        lats.extend(lat_cc.tolist()+[None]) 
        lons.extend(lon_cc.tolist()+[None])
        
       
    return lons, lats

In [6]:
# Make shortcut to Basemap object, 
# not specifying projection type for this example
map = Basemap() 

cc_lons, cc_lats=get_coastline_traces(map)
country_lons, country_lats=get_country_traces(map)

#concatenate the lon/lat for coastlines and country boundaries:
lons=cc_lons+[None]+country_lons
lats=cc_lats+[None]+country_lats

In [7]:
continent_lons = np.array(lons)
continent_lats = np.array(lats)

# get countries on cartesian map

In [18]:
XC = np.zeros(continent_lats.size)
YC = np.zeros(continent_lats.size)
ZC = np.zeros(continent_lats.size)
i=0
for lat,lon in zip(continent_lats,continent_lons):
    if lat is not None and lon is not None:
        x,y,z = get_cartesian(lat=lat,lon=lon, R=6750.)
    else:
        x = np.nan
        y = np.nan
        z = np.nan
    XC[i] = x
    YC[i] = y
    ZC[i] = z
    if i % 10000 == 0:
        print(i)
    i=i+1

0
10000
20000


# Dataset

- open dataset
- get lat, lon and TS values as numpy arrays

In [9]:
filename = "../data/v2.2.beta_mpich_gnu6.4.0_seFch16.cam.h0.1850-01_TS.nc"

In [10]:
df = xr.open_dataset(filename)
df

<xarray.Dataset>
Dimensions:           (ilev: 33, lev: 32, nbnd: 2, ncol: 48602, time: 1)
Coordinates:
  * lev               (lev) float64 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * ilev              (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time              (time) object 1850-02-01 00:00:00
Dimensions without coordinates: nbnd, ncol
Data variables:
    lat               (ncol) float64 ...
    lon               (ncol) float64 ...
    area              (ncol) float64 ...
    hyam              (lev) float64 ...
    hybm              (lev) float64 ...
    hyai              (ilev) float64 ...
    hybi              (ilev) float64 ...
    date              (time) int32 ...
    datesec           (time) int32 ...
    time_bnds         (time, nbnd) object ...
    date_written      (time) |S8 ...
    time_written      (time) |S8 ...
    ndbase            int32 ...
    nsbase            int32 ...
    nbdate            int32 ...
    nbsec             int32 ...
    mdt           

In [11]:
df.lon.shape
XS=np.zeros(df.lon.shape)
YS=np.zeros(df.lon.shape)
ZS=np.zeros(df.lon.shape)
print(df.lon.shape)

(48602,)


In [12]:
i=0
for lat,lon in zip(df.lat,df.lon):
    x,y,z = get_cartesian(lat=lat,lon=lon)
    XS[i] = x
    YS[i] = y
    ZS[i] = z
    if i % 10000 == 0:
        print(i)
    i=i+1

0
10000
20000
30000
40000


### Visualize both countries and Surface temperature

In [19]:
import ipyvolume as ipv

Z = df.TS.values.astype('float64')
from matplotlib import cm
colormap = cm.jet

znorm = Z - Z.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = colormap(znorm)
ipv.figure(width=600, height=600)
countries = ipv.plot(XC,YC,ZC, color='black', size=1)
mesh = ipv.scatter(XS, YS, ZS, color=color[...,:3])
ipv.pylab.style.box_off(), 
ipv.pylab.style.axes_off()
ipv.show()
ipv.pylab.save("ipyvolume_sphere3D.html")

# Same plot with K3D-jupyter

In [20]:
import k3d

In [21]:
XS.shape, YS.shape, ZS.shape

((48602,), (48602,), (48602,))

In [22]:
pos=np.vstack((XS,YS,ZS)).T

### Get Surface temperature values for colors

In [23]:
ZAF=Z[0,:]

In [24]:
colors = k3d.helpers.map_colors((ZAF.astype(np.float32)),  
        k3d.colormaps.basic_color_maps.CoolWarm,[210,320]).astype(np.uint32)

### Create new country contours

In [25]:
XC = np.zeros(continent_lats.size)
YC = np.zeros(continent_lats.size)
ZC = np.zeros(continent_lats.size)
i=0
for lat,lon in zip(continent_lats,continent_lons):
    if lat is not None and lon is not None:
        x,y,z = get_cartesian(lat=lat,lon=lon, R=6400) # R=6750.)
    else:
        x = np.nan
        y = np.nan
        z = np.nan
    XC[i] = x
    YC[i] = y
    ZC[i] = z
    if i % 10000 == 0:
        print(i)
    i=i+1

0
10000
20000


In [26]:
cpos=np.vstack((XC,YC,ZC)).T

In [27]:
plot = k3d.plot()
points = k3d.points(pos.astype(np.float32), colors.astype(np.uint32), point_size=150.0, shader='flat')
plot += points
# Add countries
countries = k3d.line(cpos.astype(np.float32),color=0x000000, point_size=150.0, shader='flat')
plot += countries
plot.display()

Output()

In [28]:
plot.grid_visible = False

In [ ]:
import branca.colormap as cm
linear = cm.LinearColormap(
    ['blue', 'yellow', 'red'],
    vmin=210, vmax=320
)

In [ ]:
linear